In [ ]:
import requests
from fake_useragent import UserAgent
from lxml import html
import csv
ua = UserAgent()
# url = 'https://www.moskvaonline.ru/ratesmobile/vygodnye'
url = 'https://dominternet.ru'
headers = {'User-Agent': ua.random}
session = requests.Session()

try:
    response = session.get(url + '/msk/providers-to-address/', headers=headers)
    response.raise_for_status()

    tree = html.fromstring(response.content)
    
    # Используем более надежный способ нахождения таблиц,
    # учитывая, что структура может меняться
    providers = tree.xpath('//a[@class="subhead-3"]')
    print(type(providers))
    for proveder in providers:
        provider_name = proveder.text_content()
        provider_fref = proveder.get('href')
        while True:
            try:
                response = session.get(url + provider_fref, headers=headers)
                response.raise_for_status()
                
                # tree = html.fromstring(response.content)
                print(tree)
            except:
                print('Error')
        

    if not providers:
        print("Таблица не найдена на странице.")
        exit()
    


    # first_table = tables[0] # Берем первую таблицу (если их несколько)
    # rows = first_table.xpath('./tbody/tr')
  
    # with open('тарифы.csv', 'w', newline='', encoding='utf-8') as csvfile:
    #     writer = csv.writer(csvfile)
    #     header = [th.text_content().strip() for th in first_table.xpath('./thead/tr/th')]
    #     writer.writerow(header)

    #     for row in rows:
    #         cells = row.xpath('./td')
    #         if cells:
    #             row_data = [cell.text_content().strip() for cell in cells]
    #             writer.writerow(row_data)
        

except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе: {e}")
except Exception as e:
    print(f"Ошибка при парсинге: {e}")

<class 'list'>
<Element html at 0x19f004e5d60>
<Element html at 0x19f004e7890>
<Element html at 0x19f7fd3e4e0>
<Element html at 0x19f004e54f0>
<Element html at 0x19f004e7700>
<Element html at 0x19f004e68a0>
<Element html at 0x19f004e5f40>
<Element html at 0x19f004e6210>
<Element html at 0x19f7fd3e4e0>
<Element html at 0x19f004e6940>
<Element html at 0x19f004e6990>
<Element html at 0x19f004e5db0>
<Element html at 0x19f004e5f40>
<Element html at 0x19f004e70c0>
<Element html at 0x19f004e7fc0>
<Element html at 0x19f004e6120>
<Element html at 0x19f004e6c60>
<Element html at 0x19f7fd3e4e0>
<Element html at 0x19f004e7a20>
<Element html at 0x19f004e6c60>
<Element html at 0x19f004e6940>
<Element html at 0x19f004e5f40>
<Element html at 0x19f004e6fd0>
<Element html at 0x19f004e6990>
<Element html at 0x19f004e64e0>
<Element html at 0x19f7fd3e4e0>
<Element html at 0x19f7fed3d40>
<Element html at 0x19f7fe83d90>
<Element html at 0x19f004e7a20>
<Element html at 0x19f004e7a70>
<Element html at 0x19f004

In [58]:
import requests
from fake_useragent import UserAgent
from lxml import html
import csv
import re
ua = UserAgent()
# url = 'https://www.moskvaonline.ru/ratesmobile/vygodnye'
url = 'https://megasimka.ru/product-category/sim-cards/'
headers = {'User-Agent': ua.random}
session = requests.Session()

try:
    response = session.get(url, headers=headers)
    response.raise_for_status()

    tree = html.fromstring(response.content)
    
    # Используем более надежный способ нахождения таблиц,
    # учитывая, что структура может меняться
    sims = tree.xpath('//div[contains(@class,"product-small")]//a[contains(@aria-label,"SIM")]')
    # print(len(sim_carts))
    all_sim_cards = []
    for sim_items in sims:
        num_sim_card = {}
        # provider_name = proveder.text_content().strip()
        # print(provider_name)
        sim_fref = sim_items.get('href')
        # print(sim_fref)
        # while True:
        try:
            response = session.get(sim_fref, headers=headers)
            response.raise_for_status()
            # print(response.status_code)
            tree_loc = html.fromstring(response.content)
            sim_attrs = tree_loc.xpath('.//div[@class="col-inner"]//h1')
            for sim_atrr in sim_attrs:
                # print(sim_atrr.text_content().strip())
                num_sim_card['name'] = sim_atrr.text_content().strip()
                
            prices = tree_loc.xpath('.//p[@class="price product-page-price price-on-sale"]/span')
            if len(prices) > 0:
                # for price in prices:
                # print(price.text_content().strip(?))
                price_tmp0 = prices[0].text_content().strip()
                price_tmp1 = prices[1].text_content().strip()
                num_sim_card['past_price'] = re.search(r'Первоначальная цена составляла\s*(.*)₽\.', price_tmp0)
                num_sim_card['past_price'] = num_sim_card['past_price'].group(1) if num_sim_card['past_price'] else False
                # print(num_sim_card['past_price'])
                num_sim_card['last_price'] = re.search(r'Текущая цена:\s*(.*)₽\.', price_tmp1)
                num_sim_card['last_price'] = num_sim_card['last_price'].group(1) if num_sim_card['last_price'] else False
                # print(num_sim_card['last_price'])
            else: num_sim_card['last_price'] = 'None'
            raiting = tree_loc.xpath('//div[@class="woocommerce-product-rating"]//span/strong/text()')
            if len(raiting) > 0:
                # print(raiting.pop())
                num_sim_card['raiting'] = raiting.pop()
            else: num_sim_card['raiting'] = 'None'
            description_text = tree_loc.xpath('//div[@class="product-short-description"]/p')
            if len(description_text) > 0: 
                description_text = description_text.pop().text_content()
                # Регулярные выражения для извлечения информации
                # price_sim = re.search(r'Цена eSIM:\s*(\d+₽)', description_text)
                subscr_price = re.search(r'Абонентская плата:\s*(\d+\s*₽\s*в месяц)', description_text)
                call_local = re.search(r'Исходящие звонки внутри сети:\s*(.*)', description_text)
                call_all = re.search(r'Исходящие звонки на любые номера по всей РФ:\s*(.*)', description_text)
                share_smartphone = re.search(r'Раздача со смартфона:\s*(.*)', description_text)
                sms_mms = re.search(r'SMS/MMS:\s*(.*)', description_text)
                traffic = re.search(r'Трафик:\s*(.*)', description_text)
                speed = re.search(r'Скорость:\s*(.*)', description_text)

                # Извлечение найденных значений
                # pricesim = price_sim.group(1) if price_sim else None
                num_sim_card['subscription_fee'] = subscr_price.group(1) if subscr_price else None
                num_sim_card['call_local'] = call_local.group(1) if call_local else None
                num_sim_card['call_all'] = call_all.group(1) if call_all else None
                num_sim_card['share_smartphone'] = share_smartphone.group(1) if share_smartphone else None
                num_sim_card['sms_mms'] = sms_mms.group(1) if sms_mms else None
                num_sim_card['traffic'] = traffic.group(1) if traffic else None
                num_sim_card['speed'] = speed.group(1) if speed else None
                all_sim_cards.append(num_sim_card)
                # Вывод результатов
                # print(f"Цена eSIM: {pricesim}")
                # print(f"Абонентская плата: {subscription_fee}")
                # print(f"Исходящие звонки внутри сети: {call_local}")
                # print(f"Исходящие звонки на любые номера по всей РФ: {call_all}")
                # print(f"Раздача со смартфона: {share_smartphone}")
                # print(f"SMS/MMS: {sms_mms}")
                # print(f"Трафик: {traffic}")
                # print(f"Скорость: {speed}")

        except requests.exceptions.HTTPError as err:
            print(f"HTTP error occurred: {err}")
            
        except Exception as e:
            print(f"An error occurred: {e}")
    print(all_sim_cards)
        

    if not sims:
        print("Таблица не найдена на странице.")
        exit()
    


    # first_table = tables[0] # Берем первую таблицу (если их несколько)
    # rows = first_table.xpath('./tbody/tr')
  
    # with open('тарифы.csv', 'w', newline='', encoding='utf-8') as csvfile:
    #     writer = csv.writer(csvfile)
    #     header = [th.text_content().strip() for th in first_table.xpath('./thead/tr/th')]
    #     writer.writerow(header)

    #     for row in rows:
    #         cells = row.xpath('./td')
    #         if cells:
    #             row_data = [cell.text_content().strip() for cell in cells]
    #             writer.writerow(row_data)
        

except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе: {e}")
except Exception as e:
    print(f"Ошибка при парсинге: {e}")

[{'name': 'eSIM Ростелеком (ТЕЛЕ 2) 155', 'past_price': '2,990\xa0', 'last_price': '1,190\xa0', 'raiting': 'None', 'subscription_fee': '155 ₽ в месяц', 'call_local': None, 'call_all': '800 минут в месяц', 'share_smartphone': 'Да', 'sms_mms': None, 'traffic': '10 ГБ , безлимитное использование youtube, использование приложений Яндекс.Карты, Яндекс.Навигатор и Яндекс.Транспорт на территории все России, кроме роме Республики Крым и г. Севастополя позволяет не расходовать интернет-трафик, предоставляется доступ к социальным сетям «ВКонтакте», «Одноклассники» а также мессенджерам «WhatsApp», «Viber», «ТамТам» через официальные сайты и приложения без расходования интернет-трафика.', 'speed': 'без ограничений скорости (до 100 мбит/с)'}, {'name': 'eSIM Ростелеком (ТЕЛЕ 2) 205', 'past_price': '2,990\xa0', 'last_price': '1,990\xa0', 'raiting': '5.00', 'subscription_fee': '205 ₽ в месяц', 'call_local': 'безлимитно', 'call_all': '5000 минут в месяц', 'share_smartphone': 'Да', 'sms_mms': '500 в мес